In [1]:
# Clone the repository
!git clone https://github.com/walsharry/sign-vq-transformer.git

# Move into the newly cloned directory
%cd sign-vq-transformer

Cloning into 'sign-vq-transformer'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 90 (delta 9), reused 87 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (90/90), 13.88 MiB | 40.14 MiB/s, done.
Resolving deltas: 100% (9/9), done.
/kaggle/working/sign-vq-transformer


In [2]:
# NEW Step 1.2: Install Dependencies with NumPy constraint
!pip install "numpy<2" \
             lightning==2.1.4 \
             torchmetrics \
             matplotlib \
             wandb \
             fastdtw \
             sacrebleu==2.4.0 \
             tokenizers \
             jiwer \
             fasttext \
             opencv-python \
             mediapipe \
             pycpd \
             ffmpeg \
             lmdb \
             jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 367.5 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of opencv-python to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 34.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 28.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 48.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━

In [3]:
import wandb
from kaggle_secrets import UserSecretsClient

# Get your W&B API key from Kaggle secrets
user_secrets = UserSecretsClient()
wandb_api_key = user_secrets.get_secret("WANDB_API_KEY")

# Log in to W&B non-interactively
wandb.login(key=wandb_api_key)

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

True

In [4]:
# Link the main data directory
!ln -s /kaggle/input/sign-vq-transformer-data/data /kaggle/working/sign-vq-transformer/data

In [5]:
# Create the parent 'models' directory first
!mkdir -p /kaggle/working/sign-vq-transformer/models

In [6]:
# Link the back-translation model directory
!ln -s /kaggle/input/sign-vq-transformer-data/backTranslation_PHIX_model /kaggle/working/sign-vq-transformer/models/backTranslation_PHIX_model

In [7]:
# Verify the data link
!echo "Checking data link:"
!ls -l /kaggle/working/sign-vq-transformer/data

# Verify the models directory and the back-translation model link
!echo -e "\nChecking models link:"
!ls -l /kaggle/working/sign-vq-transformer/models/

Checking data link:
lrwxrwxrwx 1 root root 43 Nov  2 11:21 /kaggle/working/sign-vq-transformer/data -> /kaggle/input/sign-vq-transformer-data/data

Checking models link:
total 4
lrwxrwxrwx 1 root root 65 Nov  2 11:21 backTranslation_PHIX_model -> /kaggle/input/sign-vq-transformer-data/backTranslation_PHIX_model


In [8]:
# Set VQ model epochs to 400
!sed -i -e 's/epochs: 1500/epochs: 400/g' -e 's/epochs: 15/epochs: 400/g' config/codebook/codebook_config.yaml

# Set translation model epochs to 400
!sed -i -e 's/epochs: 500/epochs: 400/g' -e 's/epochs: 15/epochs: 400/g' config/translation/translation_config.yaml

In [9]:
# Verify the epoch change for the VQ model
!echo "--- VQ Model Config ---"
!grep "epochs:" config/codebook/codebook_config.yaml

# Verify the epoch change for the translation model
!echo -e "\n--- Translation Model Config ---"
!grep "epochs:" config/translation/translation_config.yaml

--- VQ Model Config ---
    test_every_n_epochs: 1          # test every this many epochs, default: 1
    epochs: 400                    # train for this many epochs (will be reset in resumed process)
            epochs: [ 0, 50, 100, 1300]  # means from epoch 0-50 call every 1 epoch, from 50-100 every 5 epochs...

--- Translation Model Config ---
    test_every_n_epochs: 1          # test every this many epochs, default: 1
    epochs: 400                     # train for this many epochs (will be reset in resumed process)


In [10]:
# This command finds the buggy line in loggers.py and replaces it with a safer version.

# Original (buggy) line:
# while self.logger.hasHandlers():
#
# New (safe) line:
# while len(self.logger.handlers) > 0:

!sed -i 's/while self.logger.hasHandlers():/while len(self.logger.handlers) > 0:/g' loggers.py

In [11]:
# This command will add the necessary lines to __main__.py
# to tell PyTorch it's safe to load 'pathlib.PosixPath'
!sed -i '/import os/a import torch\nimport pathlib\ntorch.serialization.add_safe_globals([pathlib.PosixPath])' __main__.py

In [ ]:
# Run the first training step: VQ Model
!python __main__.py train vq config/codebook/codebook_config.yaml

2025-11-02 11:21:26,225 - INFO - Sign Level VQVAE Ready!!!
INFO: Sign Level VQVAE Ready!!!
2025-11-02 11:21:26,225 - INFO -                           cfg.name : phix_codebook
INFO:                           cfg.name : phix_codebook
2025-11-02 11:21:26,225 - INFO -                   cfg.wandb_logger : True
INFO:                   cfg.wandb_logger : True
2025-11-02 11:21:26,225 - INFO -                      cfg.log_level : INFO
INFO:                      cfg.log_level : INFO
2025-11-02 11:21:26,226 - INFO -                   cfg.local_logger : True
INFO:                   cfg.local_logger : True
2025-11-02 11:21:26,226 - INFO -                    cfg.save_images : True
INFO:                    cfg.save_images : True
2025-11-02 11:21:26,226 - INFO -                      cfg.save_path : ./models/vq_models/
INFO:                      cfg.save_path : ./models/vq_models/
2025-11-02 11:21:26,226 - INFO -               cfg.plot_n_sequences : 1
INFO:               cfg.plot_n_sequences : 1
2025-1

In [ ]:
# Verify that the VQ model was saved from Part 4
!ls -l models/vq_models/phix_codebook/

In [14]:
# Run the second training step: Translation Model
!python __main__.py train translate config/translation/translation_config.yaml



Running Back Translation:   0%|                          | 0/16 [00:00<?, ?it/s]

Running Back Translation:   6%|█▏                | 1/16 [00:00<00:03,  4.08it/s]

Running Back Translation:  12%|██▎               | 2/16 [00:00<00:03,  3.98it/s]

Running Back Translation:  19%|███▍              | 3/16 [00:00<00:03,  3.81it/s]

Running Back Translation:  25%|████▌             | 4/16 [00:01<00:03,  3.89it/s]

Running Back Translation:  31%|█████▋            | 5/16 [00:01<00:02,  3.73it/s]

Running Back Translation:  38%|██████▊           | 6/16 [00:01<00:02,  3.85it/s]

Running Back Translation:  44%|███████▉          | 7/16 [00:01<00:02,  3.85it/s]

Running Back Translation:  50%|█████████         | 8/16 [00:02<00:02,  3.76it/s]

Running Back Translation:  56%|██████████▏       | 9/16 [00:02<00:01,  3.61it/s]

Running Back Translation:  62%|██████████▋      | 10/16 [00:02<00:01,  3.68it/s]

Running Back Translation:  69%|███████████▋     | 11/16 [00:02<00:01,  3.91it/s]

Running Back T

In [15]:
import torch
data = torch.load("/kaggle/input/sign-vq-transformer-data/data/test.pt", map_location="cpu")
print(type(data))


<class 'dict'>


In [16]:
# import torch
# data = torch.load("/kaggle/input/sign-vq-transformer-data/data/dev.pt", map_location="cpu")
# print({k: (v[0] if isinstance(v, (list, torch.Tensor)) else v) for k, v in data.items()})


In [17]:
!pip install deep-translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.8 MB/s eta 0:00:00


In [18]:
import torch
import re
import string
from pathlib import Path
from deep_translator import GoogleTranslator

# --- Fix for UnpicklingError ---
# Add pathlib to PyTorch's safe globals
import pathlib
torch.serialization.add_safe_globals([pathlib.PosixPath])
# -------------------------------

# --- Import from the repo ---
from helpers import load_config, find_best_model
from train import make_dataset_and_model # This import isn't used, but good to have
from model_translation import Transformer
from model_vq import VQ_Transformer
from dataset_vq import make_batch as make_batch_vq
from constants import PAD_ID, EOS_ID, BOS_ID
from stitch import stitch_poses
from plot import make_pose_video
from dataset_vq import CodebookDataModule  # <-- Fix for NameError

# --- Set device ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- 1. Load VQ Model ---
print("Loading VQ model...")
vq_model_dir = Path("./models/vq_models/phix_codebook")
vq_config = load_config(vq_model_dir / "config.yaml")
vq_checkpoint_path = vq_model_dir / find_best_model(str(vq_model_dir))

# We need the dataset object to get the pose_dim
vq_dataset = CodebookDataModule(vq_config["data"], cuda=str(device), save_path=vq_model_dir)
vq_dataset.setup("test") # Need this to set input_size

# --- Fix for KeyError: Pass the whole vq_config ---
vq_model = VQ_Transformer(
    vq_config,  # Pass the entire config dict
    train_batch_size=1, dev_batch_size=1,
    dataset=None,
    input_size=vq_dataset.test.input_size,
    model_dir=vq_model_dir,
    fps=vq_dataset.test.fps,
    loggers={},
)

# Load the checkpoint (this will now work due to the pathlib fix)
vq_checkpoint = torch.load(vq_checkpoint_path, map_location=device)
vq_model.load_state_dict(vq_checkpoint["state_dict"], strict=True)
vq_model = vq_model.to(device).eval()
print("VQ model loaded successfully.")


# --- 2. Load Translation Model ---
print("Loading Translation model...")
trans_model_dir = Path("./models/translation_models/phix_translation")
trans_config = load_config(trans_model_dir / "config.yaml")
trans_checkpoint_path = trans_model_dir / find_best_model(str(trans_model_dir))

# Load the codebook poses from the VQ model
codebook_pose = vq_model.get_codebook_pose().to(device)

# Load the text vocabulary
text_vocab_path = trans_model_dir / "text_vocab.txt"
with open(text_vocab_path, 'r', encoding='utf-8') as f:
    text_vocab_list = [line.strip() for line in f.readlines()]
text_vocab = {word: i for i, word in enumerate(text_vocab_list)}

trans_model = Transformer(
    trans_config, # Pass the entire config dict
    save_path=trans_model_dir,
    train_batch_size=1, dev_batch_size=1,
    src_vocab=text_vocab,
    output_size=codebook_pose.shape[0] + 4, # 4 special tokens
    fps=vq_dataset.test.fps,
    ground_truth_text={},
    codebook_pose=codebook_pose,
)
trans_checkpoint = torch.load(trans_checkpoint_path, map_location=device)
trans_model.load_state_dict(trans_checkpoint["state_dict"], strict=True)
trans_model = trans_model.to(device).eval()
print("Translation model loaded successfully.")

Using device: cuda
Loading VQ model...


Loading data:: 100%|██████████| 641/641 [00:00<00:00, 7638.03it/s]


Loaded 7875 from test
VQ model loaded successfully.
Loading Translation model...


Decoding codebook: 100%|██████████| 32/32 [00:00<00:00, 70.69it/s]


Translation model loaded successfully.


In [19]:
# --- 3. Create Prediction Function ---

def predict_english_to_sign(english_sentence: str, output_name: str = "prediction.mp4"):
    """
    Takes an English sentence, translates to German, and generates a sign pose video.
    """
    with torch.no_grad():
        # --- Step 1: Translate English to German ---
        print(f"Translating: '{english_sentence}'")
        translator = GoogleTranslator(source='en', target='de')
        german_sentence = translator.translate(english_sentence)
        print(f"Translated to German: '{german_sentence}'")

        # --- Step 2: Pre-process and Tokenize German Text ---
        # This mirrors the pre-processing in dataset_translation.py
        text = german_sentence.lower()
        text = text.replace("-", " ")
        remove_chars = (
            string.punctuation.replace(".", "") + "„“…–’‘”‘‚´" + "0123456789€"
        )
        text = "".join(ch for ch in text if ch not in remove_chars).split()
        
        # Convert words to indices
        token_indices = [text_vocab.get(w, text_vocab["<unk>"]) for w in text]
        token_indices.append(EOS_ID) # Add End-of-Sequence token
        
        # --- Step 3: Create Batch for Model ---
        src = torch.tensor(token_indices, dtype=torch.long, device=device).unsqueeze(0) # Batch size of 1
        src_length = torch.tensor([len(token_indices)], dtype=torch.long, device=device)
        src_mask = (src != PAD_ID).unsqueeze(-2).to(device)

        # --- Step 4: Run Translation Model (Text -> VQ Tokens) ---
        print("Running translation model (Text -> VQ Tokens)...")
        model_settings = trans_config["model"]["beam_setting"]
        
        # Use greedy decoding (beam_size=1) as defined in your config
        vq_tokens = trans_model.greedy_decode(
            src=src,
            src_length=src_length,
            src_mask=src_mask,
            max_output_length=model_settings["max_output_length"],
        )
        
        # --- Step 5: Post-process VQ Tokens ---
        vq_tokens = vq_tokens.squeeze(0).cpu().numpy() # Get first (and only) item in batch
        
        # Find the first EOS token
        eos_index = (vq_tokens == EOS_ID).nonzero()
        if eos_index[0].size > 0:
            eos_index = eos_index[0][0]
            vq_tokens = vq_tokens[:eos_index]

        # Shift tokens (remove special tokens)
        vq_tokens = vq_tokens[vq_tokens >= 4] - 4
        
        if len(vq_tokens) == 0:
            print("Model predicted an empty sequence (no tokens). Cannot generate video.")
            return

        print(f"Model predicted {len(vq_tokens)} VQ tokens.")

        # --- Step 6: Convert VQ Tokens to Poses ---
        print("Converting VQ tokens to poses...")
        pred_pose = codebook_pose[vq_tokens]
        
        # Reshape to (N, window_size, joints, 3)
        window_size = vq_config["data"]["window_size"]
        pred_pose = pred_pose.reshape(pred_pose.shape[0], window_size, -1, 3)
        
        # --- Step 7: Stitch Poses and Generate Video ---
        print("Stitching poses and generating video...")
        
        # !!!!!!!!!!!! THE FIX IS HERE !!!!!!!!!!!!
        # Move tensor to CPU before passing to stitch function
        stitched_pose = stitch_poses(
            poses=pred_pose.cpu(), 
            stitch_config=trans_config["stitch"]
        )
        
        # Create video
        video_path = Path("./")
        video_path.mkdir(parents=True, exist_ok=True)
        make_pose_video(
            poses=[stitched_pose],
            names=["Prediction"],
            video_name=output_name,
            save_dir=video_path,
            fps=trans_model.fps,
            overwrite=True
        )
        print(f"Successfully created video: {video_path / output_name}")

In [20]:
# --- 4. Run Prediction ---
english_input = "Temperatures tonight will be between four and nine degrees."

predict_english_to_sign(english_input, output_name="sunny_weather.mp4")

Translating: 'Temperatures tonight will be between four and nine degrees.'
Translated to German: 'Die Temperaturen liegen heute Nacht zwischen vier und neun Grad.'
Running translation model (Text -> VQ Tokens)...
Model predicted 16 VQ tokens.
Converting VQ tokens to poses...
Stitching poses and generating video...


Plotting poses: 100%|██████████| 64/64 [00:15<00:00,  4.08it/s]


Successfully created video: sunny_weather.mp4


In [ ]:
import os
import time
from IPython.display import display
from ipywidgets import FileUpload

# Create the upload widget
uploader = FileUpload(accept='', multiple=False)
display(uploader)

print("Waiting for a file upload... (Don't close this tab)")

# Wait loop
while not uploader.value:
    time.sleep(2)

# When uploaded, extract the file
for filename, file_info in uploader.value.items():
    filepath = f"/kaggle/working/{filename}"
    with open(filepath, "wb") as f:
        f.write(file_info['content'])
    print(f"\n✅ File received: {filename} (saved to {filepath})")

print("\nYou can now process the uploaded file.")


FileUpload(value=(), description='Upload')

Waiting for a file upload... (Don't close this tab)
